In [1]:
!pip3 install pyfunctional
%matplotlib notebook

import numpy as np
import requests
from functional import seq
import os
import json
import scipy
import mcerp
import matplotlib.pyplot as plt

TEMP_MIN = 16
TEMP_MAX = 30

NORMALIZATION_SERVER = 300
NORMALIZATION_COOLING = 10000

resp = requests.get("https://api.seduce.fr/power_infrastructure/description/tree")
servers_names_raw = resp.json()['children'][0]['children'][1]['children'][1]['node'].get("children")
servers_names_raw = sorted(servers_names_raw, key=lambda x: int(x.split("-")[1]))

servers_names = seq(servers_names_raw).map(lambda x: x.replace("-", "_"))
# servers_names = servers_names.take(2)

nodes_names = servers_names + ["back_temperature"]

data_file_path = "/Users/jonathan/Documents/workspace/seduce_ml/data.json"

with open(data_file_path, "r") as data_file:
    data = json.load(data_file)

timestamps_with_all_data = None
for k, v in data.get("consumptions").items():
    if timestamps_with_all_data is None:
        timestamps_with_all_data = v.get("timestamps")
    old_length = len(timestamps_with_all_data)
    timestamps_with_all_data = [item for item in timestamps_with_all_data if item in v.get("timestamps")]
    # print("length %s => %s [%s]" % (old_length, len(timestamps_with_all_data), len(v.get("timestamps"))))

visualization_data = nodes_names \
        .map(lambda x: data.get("consumptions")[x]) \
        .map(lambda c: zip(c["timestamps"], c["means"], c["temperatures"]))\
        .map(lambda z: seq(z)
             .filter(lambda z: z[0] in timestamps_with_all_data)
             .filter(lambda z: z[2] >= TEMP_MIN and z[2] <= TEMP_MAX))

v = visualization_data.map(lambda z: seq(z)
             .map(lambda z: z[1])
             .map(lambda x: x if x is not None else 0))

temperatures = visualization_data.map(lambda z: seq(z)
             .map(lambda z: z[2])
             .map(lambda x: x if x is not None else 0))[0]

x = np.transpose(np.array(v.take(servers_names.size()).map(lambda x: x.map(lambda z: z / NORMALIZATION_SERVER).to_list()).to_list()))
y = np.array(v.drop(servers_names.size()).map(lambda x: x.map(lambda z: z / NORMALIZATION_COOLING).to_list()).to_list())[0]

KeyError: 'ecotype_1'

In [ ]:
fig, ax = plt.subplots()

colors = ['tab:blue', 'tab:orange', 'tab:green']

def ts_to_color_sorted_day(tss):
    #return "tab:blue"
    return float(tss) / np.max(indexes_int)

def ts_to_color_sorted_hour(tss):
    #return "tab:blue"
    return float(tss) % 36  / 36 #np.max(indexes_int)

def temperature_to_color(v):
    return v

cm = plt.cm.get_cmap('RdYlBu')

# Using set_dashes() to modify dashing of an existing line
x_values = np.array([np.mean(v) * NORMALIZATION_SERVER for v in x])
y_values = np.array([y * NORMALIZATION_COOLING])

color_values = [temperature_to_color(v) for v in temperatures]
scatter = ax.scatter(x_values, y_values, c=color_values, label='consumption', alpha=0.5)
fig.colorbar(scatter)